## Modules

In [1]:
import os
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
 
from datetime import datetime, timedelta
from itertools import product
from scipy.stats import norm

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

## Log-normal returns simulation

### Normal random vectors of log return & percentage return

In [2]:
miu0 = 0.12
sig0 = 0.24
k0 = 20
randVect0 = np.random.normal(miu0, sig0, k0)

print(f'Random log returns of {k0} periods:\n {randVect0}, \n')
print(f'Random percentage returns of {k0} periods:\n {np.exp(randVect0) - 1}')

Random log returns of 20 periods:
 [ 0.5177831   0.12969141  0.10444866 -0.09246233 -0.15872585  0.18639108
  0.1254533   0.45116374  0.14087741  0.35484623  0.35183421 -0.18597178
  0.13137714  0.32301367  0.41515505 -0.15249196 -0.29531534  0.03871259
 -0.13282912 -0.29904255], 

Random percentage returns of 20 periods:
 [ 0.6783029   0.138477    0.1100984  -0.08831645 -0.14676976  0.20489337
  0.13366223  0.57013836  0.15128351  0.42596136  0.42167281 -0.16970298
  0.14039778  0.38128423  0.51460557 -0.1414342  -0.25570316  0.03947169
 -0.12438529 -0.25847214]


### Fixed-leverage-betting on normally distributed returns

In [3]:
def getSim_fixLev(initAmount=100, lev=1.00, miu=0.05, sig=0.2, numPeriod=60, numSim=1000):
    """
    Obtain dataframe of fixed-leverage-bet simulations, with returns of each interval normally distributed.
    Assume zero-cost-rebalance at the end of each period.
    initAmount: initial capital
    miu: (non-annualized) mean return
    sig: (non-annualized) sigma
    numPeriod: number of periods
    numSim: number of simulations
    """
    # Dict for recording different series of total equity
    dictAmount = {}
    # Generate a total of `numSim`= N series of normally distributed returns
    for num in range(numSim):
        # vector of log returns in each period and exponentiate
        arrPct = np.exp(np.random.normal(miu, sig, numPeriod))
        # convert into growth factor vector by converting to percentage change vector, multiply by leverage, and add 1
        arrFactor = 1 + lev * (arrPct - 1)
        # equity vector by cumulative multiplying by growth factors
        arrAmount = initAmount * arrFactor.cumprod()
        # IF equity drops to 0 or even below (due to over-leverage), stop betting, set the remaining equity to 1/10000
        # of initial amount and fix it in the remaining series (for the sake of legal semi-log equity curve plotting)
        # (This artificial "residual equity" is unreal assummption, the reality is more cruel than this!)
        numBet = 1
        amtRuin = initAmount / 10000
        while numBet <= numPeriod - 1:
            if arrAmount[numBet] <= amtRuin:
                for j in range(numBet, numPeriod):
                    arrAmount[j] = amtRuin
                break
            numBet += 1
        dictAmount[f's{num + 1}'] = arrAmount
        # Form dataframe from the `dictAmount` and transpose, so that each row corresponds to a betting series
    dfSim = pd.DataFrame(dictAmount).transpose()
    # Rename columns so that each number in column labels corresponds to the k-th trial
    dfSim = dfSim.rename(columns={k: (k + 1) for k in dfSim.columns})

    return dfSim

### Sample of 20 simulations of 12 periods

In [4]:
initAmount1 = 100
lev1 = 2.00
miu1 = 0.04
sig1 = 0.16
numPeriod1 = 12
numSim1 = 20

dfSim1 = getSim_fixLev(initAmount1, lev1, miu1, sig1, numPeriod1, numSim1) 
np.round(dfSim1, 2)

,1,2,3,4,5,6,7,8,9,10,11,12
s1,85.49,91.81,87.53,105.39,127.89,138.60,176.12,180.79,226.89,363.35,298.36,342.45
s2,134.39,211.42,243.63,330.29,334.89,607.95,591.09,464.39,462.60,594.35,707.18,1152.41
s3,160.62,233.14,233.99,204.53,145.74,113.73,87.58,86.15,61.97,61.81,37.77,41.56
s4,123.81,244.18,152.40,181.77,168.17,266.08,346.15,378.49,667.29,537.98,611.04,938.70
s5,157.98,217.18,309.76,443.84,479.96,543.97,543.91,306.55,384.82,551.09,610.04,346.95
s6,133.61,108.46,110.48,146.83,174.24,180.73,232.06,203.78,299.02,287.62,345.39,240.76
s7,66.84,63.61,83.61,92.03,116.77,93.30,129.83,159.96,162.61,179.91,207.54,236.19
s8,78.19,136.85,110.49,183.52,143.56,137.09,81.28,89.65,104.78,115.02,95.69,81.34
s9,179.56,156.84,117.24,119.45,137.07,152.49,125.23,157.76,226.15,103.53,92.69,102.41
s10,62.47,76.14,55.80,52.56,103.60,153.88,114.45,84.92,65.34,112.72,123.93,168.33


## Equity curve plotting

### Simulation KPIs

In [5]:
def getSimKPI_fixLev(initamt=100, lev=1.00, miu=0.05, sig=0.2, numPeriod=60, numsims=1000):
    """Obtain a dictionary of final performance KPI of simulations. """
    dfSim = getSim_fixLev(initamt, lev, miu, sig, numPeriod, numsims)
    dictKPI = {}
    dictKPI['miu'] = miu
    dictKPI['sig'] = sig
    dictKPI['lev'] = lev
    dictKPI['NAsharpe'] = round(miu * (numPeriod ** 0.5) / sig, 4)
    dictKPI['win%'] = round(100 * dfSim[dfSim[numPeriod] >= initamt].shape[0] / numsims, 2)
    dictKPI['amountAvg'] = round(dfSim[numPeriod].mean(), 2)
    dictKPI['amountMed'] = round(dfSim[numPeriod].median(), 2)
    dictKPI['amountStd'] = round(dfSim[numPeriod].std(), 2)

    return dfSim, dictKPI

### Sample 1 (same parameter as above)

In [7]:
initAmount1 = 100
lev1 = 2.00
miu1 = 0.03
sig1 = 0.15
numPeriod1 = 36
numSim1 = 1000

dfSim1, dictKPI1 = getSimKPI_fixLev(initAmount1, lev1, miu1, sig1, numPeriod1, numSim1) 
dictKPI1

{'miu': 0.03,
 'sig': 0.15,
 'lev': 2.0,
 'NAsharpe': 1.2,
 'win%': 76.6,
 'amountAvg': 1649.16,
 'amountMed': 416.72,
 'amountStd': 3922.63}

### Function

In [10]:
def plotSim_fixLev(initAmount=100, lev=1.00, miu=0.05, sig=0.2, numPeriod=60, numSim=1000):
    """Plot equity curves from the betting simulations."""
    # Dataframe and KPI dict from `getSimKPI()` function
    dfSim, dictKPI = getSimKPI_fixLev(initAmount, lev, miu, sig, numPeriod, numSim)
    # First line of diagram title
    title = f'{numSim} simulations of {numPeriod}-periods \n'
    title += f'leverage {round(lev, 4)}, miu {round(100*miu, 2)}%, sig {round(100*sig, 2)}%\n'
    # Percentage of final equity above or equal to initial amount
    winrate = dictKPI['win%']
    amountAvg = dictKPI['amountAvg'] # Arithmetic mean of final equity
    amountMed = dictKPI['amountMed'] # Median of final equity
    amountStd = dictKPI['amountStd'] # SD of final equity
    # Attach key stats to title text
    sharpe = dictKPI['NAsharpe']
    title += f'NA-sharpe: {sharpe}, final winrate: {winrate}% \n'
    title += f'Final equity mean: {amountAvg}, median: {amountMed}, SD: {amountStd}'
    # Plot figure
    fig = plt.figure(figsize=(12, 9))
    # Append title
    fig.suptitle(title, fontsize=12)
    # Transpose the equity dataframe to plot
    dfplot = dfSim.transpose()
    # Adopt semi-log scale for fixed-percent-betting & linear scale for fixed-amount-betting
    plt.semilogy(dfplot)
    plt.plot(dfplot.index, np.repeat(initAmount, numPeriod), color='black', linewidth=3, linestyle='dashed')
    plt.show()

### Plotting simulations of sample 1

In [ ]:
plotSim_fixLev(initAmount1, lev1, miu1, sig1, numPeriod1, numSim1)

### Reduce  `sig` with everything else unchanged

In [ ]:
sig2 = 0.1
plotsimcurves(initamt1, lev1, miu1, sig2, numperiods1, numsims1)

### Raise both `miu` & `sig` but keeps NA-sharpe and leverage unchanged

In [ ]:
sharpe1 = 1.2 / (numperiods1 ** 0.5)
miu3 = 0.05
sig3 = sig1 * miu3 / miu1

plotsimcurves(initamt1, lev1, miu3, sig3, numperiods1, numsims1)

### Function to check MDD

In [ ]:
def pctmdd(dfbet, mddlevels=(0.25, 0.5, 0.75)):
    """Obtain percentage of simulations with final amount dropping below the given equity threshold."""
    # Transpose the simulation dataframe
    dfbett = dfbet.transpose()
    numtrials = dfbet.shape[1]
    numsim = dfbet.shape[0]
    # Compute MDD
    dfmdd = pd.DataFrame(columns=dfbett.columns)
    for col in dfbett.columns:
        dfmdd[col] = (dfbett[col] / dfbett[col].cummax() - 1).cummin()
    # Tranpose back to match original shape
    dfmdd = dfmdd.transpose()
    # Compute percentage of simulations having MDD greater than the levels
    mdddict = {}
    for value in mddlevels:
        mdddict[value] = dfmdd[dfmdd[numtrials] <= -value].shape[0] / numsim

    
    return dfmdd, mdddict   

### Using example 1

In [ ]:
dfmdd1, mdddict1 = pctmdd(dfbet1)
for key, value in mdddict1.items():
    print(f'\n Percentage of betting series suffering {round(100*key)}% MDD: {round(100*value, 2)}%')

### Using example 3

In [ ]:
dfbet3, kpidict3 = getsimkpi(initamt1, lev1, miu3, sig3, numperiods1, numsims1) 

dfmdd3, mdddict3 = pctmdd(dfbet3)
for key, value in mdddict3.items():
    print(f'\n Percentage of betting series suffering {round(100*key)}% MDD: {round(100*value, 2)}%')

## Optimal leverage (Kelly formula of continuous returns)

### Expected geometric growth (for normally distributed returns)

- Reference: Edward Thorp, The Kelly Criterion in Blackjack, Sports Betting, and the Stock Market (1997)

In [ ]:
def getnormgrow(lev=1.00, miu=0.03, sig=0.15, n=1):
    """Obtain expected geometric returns of random walk returns."""
    growth = n * lev * (miu - sig**2 * lev / 2)
    
    return growth

def plotlevgrow(miu=0.03, sig=0.15, n=1, levmin=0.20, levmax=4.0, step=0.001):
    """Plot expected geometric growth given miu & sigma, and identify optimal leverage."""
    # Form pandas Series of expected return of varying leverage
    levarr = np.arange(levmin, levmax, step)
    growdict = {lev: getnormgrow(lev, miu, sig, n) for lev in levarr}
    dfgrow = pd.DataFrame(pd.Series(growdict))
    # Plot interactive diagram of f-percent curve
    fig = px.line(x=dfgrow.index, y=dfgrow[0], labels={'x': 'lev', 'y': f'growth factor on {n} periods'})
    fig.add_trace(go.Scatter(x=dfgrow.index, y=np.repeat(0, len(levarr)), name='0%'))
    # Identify optimal leverage
    bestlev = round(dfgrow[0].idxmax(), 4)
    bestpl = round(dfgrow[0].max(), 4)
    fig.add_trace(go.Scatter(x=(bestlev, ), y=(bestpl, ), line_color='green', name='Opt-lev', mode='markers+text', 
                             marker_size=10, text=f'{bestlev, round(bestpl, 4)}', textposition='bottom center'))
    # Title
    fig.update_layout(title=f'Expected geometric growoth of N({miu}, {sig})', title_x=0.5, width=1000, height=500)
    # Show diagram
    fig.show()

In [ ]:
getnormgrow(lev1, miu1, sig1, numperiods1)

In [ ]:
plotlevgrow(miu1, sig1, numperiods1)

In [ ]:
plotlevgrow(miu1, sig2, numperiods1, 1.0, 5.0)

In [ ]:
plotlevgrow(miu3, sig3, numperiods1, 0.2, 2.0)

### Kelly optimal leverage (for normally distributed returns ONLY)

In [ ]:
def getkellylev(miu, sig):
    """Obtain Kelly formula of optimal leverage."""
    return round(max(miu / sig**2, 0), 4)

### Kelly leverage for a bunch of (`miu`, `sig`) pair

- reasonable range of monthly returns: 0.5% - 5%  (annualized log-returns: 6% - 60%)
- reasonable range of monthly volatility: 5% - 36%  (annualized volatility:  18% - 126%)

In [ ]:
miuarr = np.arange(0.5, 5.5, 0.5) / 100
sigarr = np.arange(0.18, 1.38, 0.12) / (12**0.5)
nummonths = 36

dfoptlev = pd.DataFrame()
for miu, sig in product(miuarr, sigarr):
    dfoptlev.loc[f'{round(100*miu, 2)}%', f'{round(100*sig, 2)}%'] = round(miu / sig**2, 4)
    
dfoptlev

### NA-sharpe for a bunch of (`miu`, `sig`) pair

In [ ]:
dfsharpe = pd.DataFrame()

for miu, sig in product(miuarr, sigarr):
    dfsharpe.loc[f'{round(100*miu, 2)}%', f'{round(100*sig, 2)}%'] = round(miu / sig * (nummonths ** 0.5), 4)
    
dfsharpe

### Expected final equity on optimal leverage

In [ ]:
dfpl = pd.DataFrame()

for miu, sig in product(miuarr, sigarr):
    optlev = dfoptlev.loc[f'{round(100*miu, 2)}%', f'{round(100*sig, 2)}%']
    dfpl.loc[f'{round(100*miu, 2)}%', f'{round(100*sig, 2)}%'] = round(100 * np.exp(getnormgrow(optlev, miu, sig, nummonths)), 2)
    
dfpl

## Adopt real stock price and check historical results on using leverage

### Collecting OHLC data

In [ ]:
import yfinance as yf

def getyahoodata(symbollist, adjust=True, startstr='1990-01-01', endstr='2046-12-31'):
    """Scrape via yahoo API to obtain data for a symbollist."""
    symbolstr = ' '.join(symbollist)
    renamedict = {'Date': 'date', 'Open': 'op', 'High': 'hi', 'Low': 'lo', 'Close': 'cl', 'Volume': 'vol',
                   'Adj Close': 'adj_cl', 'Dividends': 'div', 'Stock Splits': 'split'}
    ohlcvdfield = ['op', 'hi', 'lo', 'cl', 'vol', 'div']
    datadict = {}

    try:
        dfdata = yf.download(symbolstr, start=startstr, end=endstr, auto_adjust=False, actions=True, 
                         group_by='Tickers', threads=16)
    except:
        dfdata = pd.DataFrame()

    for symbol in symbollist:
        #try:
        dfsymbol = dfdata[(symbol, )].dropna()  # Raw data for the symbol
        dfsymbol = dfsymbol[(dfsymbol['Volume'] > 0) | (dfsymbol['High'] > dfsymbol['Low'])] # Filter bad data
        dfsymbol = dfsymbol.reset_index()
        dfsymbol = dfsymbol.rename(columns=renamedict)
        dfsymbol = dfsymbol.set_index('date')
        if not adjust:
            dfsymbol = dfsymbol[ohlcvdfield]
            dfsymbol = dfsymbol.rename(columns={field: f'{symbol}_{field}' for field in ohlcvdfield})
        else:
            adjfactor = dfsymbol['adj_cl'] / dfsymbol['cl']
            for field in ohlcvdfield[:-2]:
                dfsymbol[f'adj_{field}'] = dfsymbol[field] * adjfactor
            dfsymbol['adj_vol'] = dfsymbol['vol'] / adjfactor
            dfsymbol = dfsymbol[[f'adj_{field}' for field in ohlcvdfield[:-1]]]
            dfsymbol = dfsymbol.rename(columns={f'adj_{field}': f'{symbol}_{field}' for field in ohlcvdfield[:-1]})
            dfsymbol = np.round(dfsymbol, 4)
        datadict[symbol] = dfsymbol
        #except:
        #    print(f'Failed preparing data for {symbol}.')

    dfallsymbols = pd.concat(datadict.values(), axis=1, join='inner')
    dfallsymbols = dfallsymbols.fillna(method='ffill')
    dfallsymbols = np.round(dfallsymbols, 4)

    return dfallsymbols

etflist = ['SPY', 'QQQ', 'SMH', 'ARKK', 'EWH']
chipslist = ['TSLA', 'MSFT', 'NVDA', 'AMZN', 'MSCI']
assetlist = etflist + chipslist

startstr = '2010-07-02'
endstr = '2022-07-01'
dfohlcall = getyahoodata(assetlist, True, startstr, endstr)
dfohlcall.index

### Resample into monthly OHLC

In [ ]:
def ohlcresample(dfohlc, assetlist, freq='M'):
    """Resample the OHLC dataframe into desired timeframe."""
    aggrule = {}
    for asset in assetlist:
        aggrule[f'{asset}_op'] = 'first'
        aggrule[f'{asset}_hi'] = 'max'
        aggrule[f'{asset}_lo'] = 'min'
        aggrule[f'{asset}_cl'] = 'last'
        aggrule[f'{asset}_vol'] = 'sum'
    dfnew = dfohlc.resample(rule=freq, label='right').agg(aggrule)
    
    return dfnew

dfmonthall = ohlcresample(dfohlcall, assetlist)
dfmonthall.iloc[-12:, :10]

### Compute optimal leverage of all symbols in every 36-month-period

In [ ]:
nummonths = 36
startstr1 = dfmonthall.index[-1-nummonths].strftime('%Y-%m-%d')
endstr1 = dfmonthall.index[-1].strftime('%Y-%m-%d')

dfopthist = pd.DataFrame(columns=['miu', 'sig', 'NAsharpe', 'optlev'])
dfpct = pd.DataFrame()

for asset in assetlist:
    dfmonth = dfmonthall.loc[startstr1:endstr1, [f'{asset}_cl']]
    dfpct[f'{asset}_pct'] = np.round(np.log(dfmonth[f'{asset}_cl'] / dfmonth[f'{asset}_cl'].shift(1)), 5)
    dfpct[f'{asset}_chg'] = np.round(dfmonth[f'{asset}_cl'] / dfmonth[f'{asset}_cl'].shift(1) - 1, 5)
    dfopthist.loc[asset, 'miu'] = round(dfpct[f'{asset}_pct'].mean(), 5)
    dfopthist.loc[asset, 'sig'] = round(dfpct[f'{asset}_pct'].std(), 5)
    dfopthist.loc[asset, 'NAsharpe'] = round(dfopthist.loc[asset, 'miu']/dfopthist.loc[asset, 'sig']*(nummonths**0.5), 4)
    dfopthist.loc[asset, 'optlev'] = getkellylev(dfopthist.loc[asset, 'miu'] , dfopthist.loc[asset, 'sig'])
    
dfopthist

### 1x leverage cumulative returns

In [ ]:
dfnav0 = pd.DataFrame(index=dfpct.index)

for asset in assetlist:
    dfnav0[f'{asset}_NAV'] = (1 + dfpct[f'{asset}_chg']).cumprod()
    
dfnav0.iloc[0] = 1.00

for asset in assetlist:
    dfnav0[f'{asset}_DD'] = dfnav0[f'{asset}_NAV'] / dfnav0[f'{asset}_NAV'].cummax() - 1
    dfnav0[f'{asset}_MDD'] = dfnav0[f'{asset}_DD'].cummin()
    
productfield = product(assetlist, ['NAV', 'DD', 'MDD'])
dfnav0 = dfnav0[[f'{asset}_{field}' for asset, field in productfield]]
    
dfnav0.iloc[-6:, :12]

### Append 1x return statistics to optlev dataframe

In [ ]:
for asset in assetlist:
    dfopthist.loc[asset, '1xNAV'] = dfnav0.loc[endstr1, f'{asset}_NAV']
    dfopthist.loc[asset, '1xMDD'] = dfnav0.loc[endstr1, f'{asset}_MDD']   

dfopthist

### Adopt optimal leverage to past 36 months

In [ ]:
tryassetlist = [asset for asset in assetlist if dfopthist.loc[asset, 'optlev'] > 0]
dfnav = pd.DataFrame(index=dfpct.index)

for asset in tryassetlist:
    dfnav[f'{asset}_NAV'] = (1 + dfopthist.loc[asset, 'optlev'] * dfpct[f'{asset}_chg']).cumprod()
    
dfnav.iloc[0] = 1.0

for i in range(dfnav.shape[1]):
    for j in range(dfnav.shape[0]):
        if dfnav.iloc[j, i] <= 0:
            dfnav.iloc[j:, i] = dfnav.iloc[j, i]
            
for asset in tryassetlist:
    dfnav[f'{asset}_DD'] = dfnav[f'{asset}_NAV'] / dfnav[f'{asset}_NAV'].cummax() - 1
    dfnav[f'{asset}_MDD'] = dfnav[f'{asset}_DD'].cummin()

tryproductfield = product(tryassetlist, ['NAV', 'DD', 'MDD'])
dfnav = dfnav[[f'{asset}_{field}' for asset, field in tryproductfield]]

for asset in assetlist:
    if asset in tryassetlist:
        dfopthist.loc[asset, 'optNAV'] = dfnav.loc[endstr1, f'{asset}_NAV']
        dfopthist.loc[asset, 'optMDD'] = dfnav.loc[endstr1, f'{asset}_MDD']   
    else:
        dfopthist.loc[asset, 'optNAV'] = 1.000
        dfopthist.loc[asset, 'optMDD'] = 0.000

dfopthist

### Closer inspection on day-to-day basis using opt-lev to invest

In [ ]:
dfpctday = pd.DataFrame()

for asset in assetlist:
    dfday = dfohlcall.loc[startstr1:endstr1, [f'{asset}_cl']]
    dfpctday[f'{asset}_pct'] = np.round(np.log(dfday[f'{asset}_cl'] / dfday[f'{asset}_cl'].shift(1)), 5)
    dfpctday[f'{asset}_chg'] = np.round(dfday[f'{asset}_cl'] / dfday[f'{asset}_cl'].shift(1) - 1, 5)
    
dfnav1 = pd.DataFrame(index=dfpctday.index)

for asset in assetlist:
    dfnav1[f'{asset}_NAV'] = (1 + dfopthist.loc[asset, 'optlev'] * dfpctday[f'{asset}_chg']).cumprod()
    
dfnav1.iloc[0] = 1.0

for i in range(dfnav1.shape[1]):
    for j in range(dfnav1.shape[0]):
        if dfnav1.iloc[j, i] <= 0:
            dfnav1.iloc[j:, i] = dfnav1.iloc[j, i]
            
for asset in assetlist:
    dfnav1[f'{asset}_DD'] = dfnav1[f'{asset}_NAV'] / dfnav1[f'{asset}_NAV'].cummax() - 1
    dfnav1[f'{asset}_MDD'] = dfnav1[f'{asset}_DD'].cummin()

productfield = product(assetlist, ['NAV', 'DD', 'MDD'])
dfnav1 = dfnav1[[f'{asset}_{field}' for asset, field in productfield]]
    
dfoptday = dfopthist[['miu', 'sig', 'NAsharpe', 'optlev']]
dfoptday.rename(columns={'miu': 'month_miu', 'sig': 'month_sig'}, inplace=True)

for asset in assetlist:
    dfoptday.loc[asset, 'optNAV'] = dfnav1[f'{asset}_NAV'][-1]
    dfoptday.loc[asset, 'optMDD'] = dfnav1[f'{asset}_MDD'][-1]

dfoptday

### Search opt-lev from historical returns instead of Kelly formula

In [ ]:
dfoptreal = dfopthist[['miu', 'sig', 'NAsharpe', 'optlev']]
dfoptreal.rename(columns={'miu': 'month_miu', 'sig': 'month_sig'}, inplace=True)

dfoptreal['optNAV'] = 1.000
dfoptreal['optMDD'] = 0.000

levarr = np.arange(0.2, 5.1, 0.1)

for lev in levarr:
    dfnav2 = pd.DataFrame(index=dfpctday.index)
    
    for asset in assetlist:
        dfnav2[f'{asset}_NAV'] = (1 + lev * dfpctday[f'{asset}_chg']).cumprod()
        
    dfnav2.iloc[0] = 1.0

    for i in range(dfnav2.shape[1]):
        for j in range(dfnav2.shape[0]):
            if dfnav2.iloc[j, i] <= 0:
                dfnav2.iloc[j:, i] = dfnav2.iloc[j, i]
                
    for asset in assetlist:
        dfnav2[f'{asset}_DD'] = dfnav2[f'{asset}_NAV'] / dfnav2[f'{asset}_NAV'].cummax() - 1
        dfnav2[f'{asset}_MDD'] = dfnav2[f'{asset}_DD'].cummin()
        
        if dfnav2[f'{asset}_NAV'][-1] > dfoptreal.loc[asset, 'optNAV']:
            dfoptreal.loc[asset, 'optNAV'] = dfnav2[f'{asset}_NAV'][-1]
            dfoptreal.loc[asset, 'optMDD'] = dfnav2[f'{asset}_MDD'][-1]
            dfoptreal.loc[asset, 'optlev'] = lev
            
    print(f'{round(lev, 1)}x buy-hold-rebalance done.')

In [ ]:
dfoptreal